<a href="https://colab.research.google.com/github/currencyfxjle/BL-Distribution-West/blob/main/BL_GitHub_MainCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Monday Reports, sales by rep, item, customer on weekly intervals (without invoice & credit separation)

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Octubre.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/Octubre.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

Separation of Invoice & Credit with Totals, on weekly intervals (most complete and recent code)

In [ ]:
import pandas as pd
from openpyxl.utils import get_column_letter

# Function to calculate weekly sales matrices
def calculate_matrix(df, groupby_column):
    weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN'), 'Type']).agg({'Amount': 'sum'}).reset_index()
    pivot = weekly_sales.pivot_table(values='Amount', index=[groupby_column, 'Type'], columns='Date', fill_value=0)

    # Calculate row totals and add as a column
    pivot['Total'] = pivot.sum(axis=1)

    # Move the 'Type' level of the index to columns to separate invoices and credits
    pivot = pivot.unstack(level='Type').swaplevel(axis=1).sort_index(axis=1)

    # Correctly format the column headers as strings
    pivot.columns = [f'{col[0].strftime("%Y-%m-%d")} {col[1]}' if isinstance(col[0], pd.Timestamp)
                     else f'{col[0]} {col[1]}'
                     for col in pivot.columns.values]

    return pivot

# Define the file paths
input_file_path = '/content/week_sales.xlsx'  # Update this path to your file location
output_file_path = 'output_file.xlsx'

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile(input_file_path)

# Create an ExcelWriter to save the results
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(input_file_path, sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Calculate weekly sales matrices
        matrix_rep_df = calculate_matrix(df, 'Rep')
        matrix_item_df = calculate_matrix(df, 'Item')
        matrix_name_df = calculate_matrix(df, 'Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.to_excel(writer, sheet_name=f'{sheet_name} Rep', index=True)
        matrix_item_df.to_excel(writer, sheet_name=f'{sheet_name} Item', index=True)
        matrix_name_df.to_excel(writer, sheet_name=f'{sheet_name} Customer', index=True)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column in range(1, worksheet.max_column + 1):
                max_length = 0
                column = get_column_letter(column)
                for cell in worksheet[column]:
                    try:  # Necessary to avoid error on empty cells
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column].width = adjusted_width

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


Separation of Rep sales by Customer & Item tab in excel

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create a new Excel writer object
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the current tab as a DataFrame
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Convert 'Type', 'Name', 'Item', and 'Date' columns to appropriate datatypes
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])

        # Clean 'Amount' and 'Qty' columns by removing commas and converting to float
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # General weekly sales
        weekly_sales = df.groupby(['Rep', pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum', 'Qty': 'sum'}).reset_index()

        # Compute and append totals for weekly_sales
        totals_row = weekly_sales[['Amount', 'Qty']].sum()
        totals_row['Rep'] = 'TOTAL'
        totals_row['Date'] = 'TOTAL'
        weekly_sales = pd.concat([weekly_sales, pd.DataFrame([totals_row])], ignore_index=True)
        weekly_sales.to_excel(writer, sheet_name=sheet_name + ' - Weekly Sales', index=False)

        # For each rep, pivot the data to get weekly sales by Item and by Client
        for rep in df['Rep'].unique():
            if pd.notna(rep):  # Ignore NaN values
                rep_data = df[df['Rep'] == rep]

                # Pivot the data by Item
                pivot_data = rep_data.pivot_table(values='Amount', index='Item', columns=pd.Grouper(key='Date', freq='W-SUN'), aggfunc='sum').reset_index()

                # Calculate totals and append to the top for Item data
                totals = pivot_data.sum(numeric_only=True)
                totals['Item'] = 'TOTAL'
                pivot_with_totals = pd.concat([pd.DataFrame(totals).T, pivot_data], ignore_index=True)

                # Ensure 'Item' column is the first column
                ordered_cols = ['Item'] + [col for col in pivot_with_totals if col != 'Item']
                pivot_with_totals = pivot_with_totals[ordered_cols]

                # Write Item data to Excel
                pivot_with_totals.to_excel(writer, sheet_name=sheet_name + ' - ' + str(rep), index=False)

                # Pivot the data by Client (Name)
                pivot_client_data = rep_data.pivot_table(values='Amount', index='Name', columns=pd.Grouper(key='Date', freq='W-SUN'), aggfunc='sum').reset_index()

                # Calculate totals and append to the top for Client data
                client_totals = pivot_client_data.sum(numeric_only=True)
                client_totals['Name'] = 'TOTAL'
                pivot_client_with_totals = pd.concat([pd.DataFrame(client_totals).T, pivot_client_data], ignore_index=True)

                # Ensure 'Name' column is the first column
                ordered_client_cols = ['Name'] + [col for col in pivot_client_with_totals if col != 'Name']
                pivot_client_with_totals = pivot_client_with_totals[ordered_client_cols]

                # Write Client data to Excel
                pivot_client_with_totals.to_excel(writer, sheet_name=sheet_name + ' - ' + str(rep) + ' - Client Sales', index=False)

    # Autofit columns for each sheet
    workbook = writer.book
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')